In [ ]:
# --- Environment Bootstrapping ---import subprocessimport sysimport osfrom pathlib import Path# Clone repository if not already presentif not Path('PinokioCloud').exists():    print("Cloning PinokioCloud repository...")    subprocess.run(        ["git", "clone", "https://github.com/redfredstar/SD-Pinokibro.git", "PinokioCloud"],        check=True    )# Change to project directoryos.chdir('PinokioCloud')sys.path.append(os.getcwd())# Install required dependenciesdependencies = [    'ipywidgets',    'psutil',    'requests',    'pyngrok',    'GPUtil']for dep in dependencies:    subprocess.run([sys.executable, "-m", "pip", "install", dep], check=True)print("✅ Repository and dependencies ready")# --- Module Imports ---import ipywidgets as widgetsfrom IPython.display import displayfrom datetime import datetimeimport queueimport threadingimport tracebackfrom app.utils.P01_CloudDetector import CloudDetectorfrom app.utils.P01_PathMapper import PathMapperfrom app.core.P02_ProcessManager import ProcessManagerfrom app.utils.P03_Translator import Translatorfrom app.core.P04_EnvironmentManager import EnvironmentManagerfrom app.core.P05_SearchEngine import SearchEnginefrom app.utils.P05_AppAnalyzer import AppAnalyzerfrom app.core.P08_StateManager import P08_StateManagerfrom app.core.P13_LaunchManager import P13_LaunchManagerfrom app.utils.P14_WebUIDetector import P14_WebUIDetectorfrom app.core.P14_TunnelManager import P14_TunnelManagerfrom app.core.P11_LibraryManager import P11_LibraryManagerfrom app.core.P07_InstallManager import P07_InstallManagerprint("🚀 Initializing PinokioCloud Centralized UI Orchestrator...")# --- Core Engine Instantiation ---cloud_detector = CloudDetector()platform_info = cloud_detector.detect_platform()path_mapper = PathMapper(platform_info)process_manager = ProcessManager()state_manager = P08_StateManager(path_mapper)file_manager = P08_FileManager(path_mapper)translator = Translator()search_engine = SearchEngine(    db_path=path_mapper.get_data_path() / "cleaned_pinokio_apps.json")env_manager = EnvironmentManager(platform_info, process_manager, path_mapper)install_manager = P07_InstallManager(    process_manager, env_manager, file_manager, state_manager)library_manager = P11_LibraryManager(    state_manager, env_manager, path_mapper, process_manager)web_ui_detector = P14_WebUIDetector()def stream_to_terminal(message):    """    Stream message to terminal output widget.    """    with terminal_output:        display(            widgets.HTML(                f'<pre style="color: #333; font-family: monospace; white-space: pre-wrap; word-wrap: break-word;">{message}</pre>'            )        )tunnel_manager = P14_TunnelManager(callback=stream_to_terminal)launch_manager = P13_LaunchManager(    state_manager, translator, env_manager, process_manager, web_ui_detector, tunnel_manager)print("✅ All engines initialized successfully")# --- UI Widget Creation ---discover_output = widgets.Output()my_library_output = widgets.Output()active_tunnels_output = widgets.Output()terminal_output = widgets.Output()tab_widget = widgets.Tab()tab_widget.children = [discover_output, my_library_output, terminal_output, active_tunnels_output]tab_widget.titles = ['Discover', 'My Library', 'Terminal', 'Active Tunnels']# --- Centralized UI Orchestrator ---job_queue = queue.Queue()def _job_worker():    """    Single worker thread that processes all user actions serially.    This function runs in an infinite loop, processing jobs from the queue    one at a time to prevent race conditions and ensure UI consistency.    """    while True:        try:            # Get job from queue (blocks until job is available)            job = job_queue.get()            action, app_name = job            # Process the job based on action type            if action == 'install':                try:                    install_manager.install_app(app_name, callback=stream_to_terminal)                except Exception as e:                    error_message = (                        f"Installation failed for {app_name}: {str(e)}\n{traceback.format_exc()}"                    )                    stream_to_terminal(error_message)                    state_manager.set_app_status(app_name, 'ERROR', error_message=error_message)            elif action == 'start':                try:                    launch_manager.launch_app(                        app_name=app_name,                        primary_callback=stream_to_terminal,                        secondary_callback=lambda line: stream_to_terminal(f"Public URL: {line}"),                    )                except Exception as e:                    error_message = (                        f"Launch failed for {app_name}: {str(e)}\n{traceback.format_exc()}"                    )                    stream_to_terminal(error_message)                    state_manager.set_app_status(app_name, 'ERROR', error_message=error_message)            elif action == 'stop':                try:                    launch_manager.stop_app(app_name, callback=stream_to_terminal)                except Exception as e:                    error_message = (                        f"Stop failed for {app_name}: {str(e)}\n{traceback.format_exc()}"                    )                    stream_to_terminal(error_message)                    state_manager.set_app_status(app_name, 'ERROR', error_message=error_message)            elif action == 'certify':                try:                    app_details = library_manager.get_app_details(app_name)                    current_status = app_details.get('is_certified', False)                    library_manager.certify_app(app_name, not current_status)                except Exception as e:                    error_message = (                        f"Certification update failed for {app_name}: {str(e)}\n{traceback.format_exc()}"                    )                    stream_to_terminal(error_message)            elif action == 'uninstall':                try:                    library_manager.uninstall_app(app_name, callback=stream_to_terminal)                except Exception as e:                    error_message = (                        f"Uninstallation failed for {app_name}: {str(e)}\n{traceback.format_exc()}"                    )                    stream_to_terminal(error_message)                    state_manager.set_app_status(app_name, 'ERROR', error_message=error_message)        except Exception as e:            # Outer error boundary for any unexpected errors            error_message = f"Worker thread error: {str(e)}\n{traceback.format_exc()}"            stream_to_terminal(error_message)        finally:            # Always refresh UI and mark task as done            refresh_ui()            job_queue.task_done()# Start the single worker threadworker_thread = threading.Thread(target=_job_worker, daemon=True)worker_thread.start()# --- UI Logic and Handlers ---def on_install_click(app_name):    """    Simplified install handler - just queues the job.    """    refresh_ui(busy=True)    job_queue.put(('install', app_name))def on_start_click(app_name):    """    Simplified start handler - just queues the job.    """    refresh_ui(busy=True)    job_queue.put(('start', app_name))def on_stop_click(app_name):    """    Simplified stop handler - just queues the job.    """    refresh_ui(busy=True)    job_queue.put(('stop', app_name))def on_certify_click(app_name):    """    Simplified certify handler - just queues the job.    """    refresh_ui(busy=True)    job_queue.put(('certify', app_name))def on_uninstall_click(app_name):    """    Simplified uninstall handler - just queues the job.    """    refresh_ui(busy=True)    job_queue.put(('uninstall', app_name))def refresh_ui(busy=False):    """    Master UI refresh function - single source of truth for entire interface.    Queries the database and redraws all UI components based on current state.    When busy=True, disables all action buttons to provide user feedback.    Args:        busy: If True, disables all action buttons during background operations    """    try:        # Clear existing content from all tabs        discover_output.clear_output()        my_library_output.clear_output()        active_tunnels_output.clear_output()        terminal_output.clear_output()        # Get current application states from database        apps = state_manager.get_all_apps()        # === DISCOVER TAB ===        with discover_output:            display(widgets.HTML("<h3>Discover Applications</h3>"))            # Load application data            if search_engine.load_data():                available_apps = search_engine.search_apps('', limit=20)                for app in available_apps:                    app_name = app['name']                    # Check if app is already installed                    installed_app = next((a for a in apps if a['app_name'] == app_name), None)                    if installed_app:                        # App is installed - show status and appropriate actions                        status = installed_app['status']                        is_certified = installed_app.get('is_certified', False)                        cert_badge = " ✅ Certified" if is_certified else ""                        display(                            widgets.HTML(                                f"<b>{app_name}</b>{cert_badge}<br>Status: {status}<br><br>"                            )                        )                        if status == 'INSTALLED':                            start_button = widgets.Button(                                description=f"Start {app_name}",                                disabled=busy,                            )                            start_button.on_click(lambda b, name=app_name: on_start_click(name))                            display(start_button)                            uninstall_button = widgets.Button(                                description=f"Uninstall {app_name}",                                button_style='danger',                                disabled=busy,                            )                            uninstall_button.on_click(lambda b, name=app_name: on_uninstall_click(name))                            display(uninstall_button)                        elif status == 'RUNNING':                            stop_button = widgets.Button(                                description=f"Stop {app_name}",                                button_style='danger',                                disabled=busy,                            )                            stop_button.on_click(lambda b, name=app_name: on_stop_click(name))                            display(stop_button)                            certify_button = widgets.Button(                                description=                                    "Mark as Certified" if not is_certified else "Remove Certification",                                button_style='info' if not is_certified else 'warning',                                disabled=busy,                            )                            certify_button.on_click(lambda b, name=app_name: on_certify_click(name))                            display(certify_button)                    else:                        # App is not installed - show install button                        install_button = widgets.Button(                            description=f"Install {app_name}",                            button_style='success',                            disabled=busy,                        )                        install_button.on_click(lambda b, name=app_name: on_install_click(name))                        display(install_button)                    display(widgets.HTML("<hr>"))        # === MY LIBRARY TAB ===        with my_library_output:            display(widgets.HTML("<h3>My Library</h3>"))            # Filter for installed applications            installed_apps = [app for app in apps if app['status'] != 'NOT_INSTALLED']            if not installed_apps:                display(                    widgets.HTML(                        "No installed applications. Use the 'Discover' tab to install applications."                    )                )            else:                for app in installed_apps:                    app_name = app['app_name']                    status = app['status']                    is_certified = app.get('is_certified', False)                    cert_badge = " ✅ Certified" if is_certified else ""                    display(                        widgets.HTML(                            f"<b>{app_name}</b>{cert_badge}<br>Status: {status}<br><br>"                        )                    )                    if status == 'INSTALLED':                        start_button = widgets.Button(                            description=f"Start {app_name}",                            disabled=busy,                        )                        start_button.on_click(lambda b, name=app_name: on_start_click(name))                        display(start_button)                        uninstall_button = widgets.Button(                            description=f"Uninstall {app_name}",                            button_style='danger',                            disabled=busy,                        )                        uninstall_button.on_click(lambda b, name=app_name: on_uninstall_click(name))                        display(uninstall_button)                    elif status == 'RUNNING':                        stop_button = widgets.Button(                            description=f"Stop {app_name}",                            button_style='danger',                            disabled=busy,                        )                        stop_button.on_click(lambda b, name=app_name: on_stop_click(name))                        display(stop_button)                        certify_button = widgets.Button(                            description=                                "Mark as Certified" if not is_certified else "Remove Certification",                            button_style='info' if not is_certified else 'warning',                            disabled=busy,                        )                        certify_button.on_click(lambda b, name=app_name: on_certify_click(name))                        display(certify_button)                    display(widgets.HTML("<hr>"))        # === ACTIVE TUNNELS TAB ===        with active_tunnels_output:            display(widgets.HTML("<h3>Active Tunnels</h3>"))            # Filter for running applications with tunnel URLs            running_apps_with_tunnels = [                app for app in apps                if app['status'] == 'RUNNING' and app.get('tunnel_url', '').strip()            ]            if running_apps_with_tunnels:                for app in running_apps_with_tunnels:                    app_name = app['app_name']                    tunnel_url = app['tunnel_url']                    link_widget = widgets.HTML(                        value=f'<a href="{tunnel_url}" target="_blank" style="color: #007acc; text-decoration: none;">'                              f'🔗 {app_name} - {tunnel_url}</a><br>'                    )                    display(link_widget)            else:                display(                    widgets.HTML(                        "No active tunnels. Start an application to see tunnel links here."                    )                )        # === TERMINAL TAB ===        with terminal_output:            display(                widgets.HTML(                    "Terminal output will appear here during application operations."                )            )    except Exception as e:        # Full traceback per Maximum Debug Philosophy        error_message = f"Error refreshing UI: {str(e)}\n{traceback.format_exc()}"        with terminal_output:            display(widgets.HTML(f'<pre style="color: red;">{error_message}</pre>'))# --- Final Execution ---refresh_ui()print("✅ PinokioCloud Centralized UI Orchestrator initialized")display(tab_widget)